# Deploy MOSTLY AI SDK Algorithm Model Package from AWS Marketplace

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 



---



The MostlyAI Synthetic Data Generator is a powerful tool that enables you to create high-quality synthetic data while preserving the statistical properties and relationships of your original data. This synthetic data generation process ensures privacy compliance while maintaining data utility for various use cases including testing, development, and machine learning model training.


> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to MOSTLY AI SDK Algorithm. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> MOSTLY AI SDK Algorithm(Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
import base64
import json
import uuid
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
from urllib.parse import urlparse
import pandas as pd
from sagemaker.algorithm import AlgorithmEstimator
from sagemaker.inputs import TrainingInput
import time

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

In [ ]:
# MOSTLY AI SDK Algorithm
algorithm_arn = f"arn:aws:sagemaker:{region}:446921602837:algorithm/mostlyai-sdk-sagemaker-cpu-87738f64e93b3f59aa191d36e4fb5ed0"


In [ ]:

# Define the S3 URI to your training data
input_data_uri = "s3://testbucketuser-bucket/sagemaker-tests/train/"  

# Initialize the AlgorithmEstimator
estimator = AlgorithmEstimator(
    algorithm_arn=algorithm_arn,
    role=role,
    instance_count=1,
    instance_type="ml.c5.4xlarge",  # Or another instance type supported by your algorithm
    input_mode="File",
    sagemaker_session=sagemaker_session,
)

# Optional: set hyperparameters if your algorithm requires any
estimator.set_hyperparameters(**{
     "configFile": "generator.json",
     "generatorFile": "generator.zip"
 })

# Define the training input
train_input = TrainingInput(
    s3_data=input_data_uri,
    content_type="text/csv",  # Adjust to what your algorithm expects
)

# Start the training job
time_postfix = f"{int(time.time())}"
job_name = f"mostlyai-sdk-training-example-{time_postfix}"
estimator.fit({"train": train_input}, job_name=job_name)

model_name = f"mostlyai-sdk-model-example-{time_postfix}"

model = estimator.create_model(name=model_name)


## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Create an endpoint

In [ ]:
# create a deployable model 
endpoint_name = f"mostlyai-sdk-endpoint-example-{time_postfix}"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.4xlarge",
    endpoint_name=endpoint_name
)


Once endpoint has been created, you would be able to perform real-time inference.

<Add code snippet that shows the payload contents>

### B. Perform real-time inference

In [ ]:
# Path to your JSON file
config_path = "inference_config.json"

# Read and parse the JSON
with open(config_path, "r") as f:
    inference_config = json.load(f)

# Print or use the loaded config
print(inference_config)

In [ ]:
# Set appropriate content type — adjust if needed
predictor.content_type = "application/json"

# Convert the inference_config dict to JSON string
payload = json.dumps(inference_config)

# Make the prediction call
response_str = predictor.predict(payload)
response = json.loads(response_str)

# Print the response
print(response)

### C. Visualize output

In [ ]:
def visualize_output(response):
    message = response.get("message")
    print(f"📌 Message: {message}")
    result = response.get("result", {})
    if not result:
        print("No 'result' key found or it's empty.")
    else:
        for key, records in result.items():
            print(f"📌 Table: {key}")
            try:
                df = pd.DataFrame(records)
                display(df)
            except Exception as e:
                print(f"Could not display table for '{key}': {e}")

# Visualize each item under 'result' as a separate table
visualize_output(response)

### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint_config(endpoint_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
# Run the batch-transform job
batch_job_name = f"mostlyai-sdk-batchjob-example-{time_postfix}"

s3 = boto3.client("s3")
output_s3 = f"batch-output/{time_postfix}/"
s3.put_object(Bucket=bucket, Key=output_s3)

transformer = model.transformer(1, "ml.c5.4xlarge", output_path=f"s3://{bucket}/{output_s3}")
transformer.transform("s3://testbucketuser-bucket/sagemaker-tests/inference/", content_type="application/json",job_name=batch_job_name)
transformer.wait()

In [ ]:
# output is available on following path
output_path = transformer.output_path

# Parse the S3 path
parsed = urlparse(output_path)
bucket_name = parsed.netloc
key = parsed.path.lstrip("/")

# Download the file content
s3 = boto3.client("s3")
response_obj = s3.get_object(Bucket=bucket_name, Key=key + "inference_config.json.out")
response_body = response_obj["Body"].read().decode("utf-8")

# Parse JSON content
batch_response = json.loads(response_body)

visualize_output(batch_response)

## 4. Clean-up

### A. Delete the endpoint and the model

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name)
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

